# Hyperparameter Tuning Tutorial

This notebook provides a tutorial for tuning the hyperparameters of one of the learners from OpenDr.
In this tutorial we will go through the process of doing this for the DetrLearner, but the process will be similar for any of the other learners. The hyperparameter tuning tool is based on the (Optuna hyperparameter tuning framework).


First, we need to import the *HyperparameterTuner* and the learner for which we would like to perform hyperparameter tuning:

In [1]:
from opendr.utils.hyperparameter_tuner import HyperparameterTuner
from opendr.perception.object_detection_2d import DetrLearner

Next, we will download a dummy dataset which consists of a single image.
We are doing this for the sake of time, such that we can see improvement within a reasonable amount of time.
The procedure would be the same for a larger dataset, but then the training and evaluation time will be longer of course.

In [2]:
import os
from opendr.engine.datasets import ExternalDataset

learner = DetrLearner()
learner.download(mode='test_data')
dataset = ExternalDataset(path=os.path.join('temp', 'nano_coco'), dataset_type='COCO')

The *HyperparameterTuner* will execute the *init*, *fit* and *eval* methods of the learner a couple of times, in
order to see what the performance is of the learner with a certain set of values for the hyperparameters.
Therefore, we need to specify with which arguments these functions should be called.
These arguments are specific for the learner you are tuning, so please check the documentation of the learner_class.
We call these the *init_arguments*, *fit_arguments* and *eval_arguments*.
Note that for *init_arguments*, you should not specify the arguments that correspond to hyperparameters you want to tune. 

In [3]:
# Specify the arguments that are required for the init method
init_arguments = {'device': 'cuda'}

# Specify the arguments that are required for the fit method
fit_arguments = {
    'dataset': dataset,
    'annotations_folder': '',
    'train_annotations_file': 'instances.json',
    'train_images_folder': 'image',
    'silent': True,
}

# Specify the arguments that are required for the eval method
eval_arguments = {
    'dataset': dataset,
    'images_folder': 'image',
    'annotations_folder': '',
    'annotations_file': 'instances.json',
    'verbose': False,
}

We will now initialize the tuner. Note that we are providing the tuner with the learner_class and not with an intialized learner object.

In [4]:
tuner = HyperparameterTuner(DetrLearner)

[I 2021-11-01 13:56:44,034] A new study created in memory with name: no-name-1affa871-c7c6-46e6-8c5a-78822faa22fe


No Study object is provided, a default one will be created using optuna.create_study().
The get_hyperparameters method is implemented in the DetrLearner class.
The optimize method can be run without specifying the hyperparameters argument.
If the hyperparameters argument is not specified for the optimize method, the following hyperparameters will be tuned:
name       type         choices                      low    high  log
---------  -----------  -------------------------  -----  ------  -----
optimizer  categorical  ['sgd', 'adam', 'adamw']
backbone   categorical  ['resnet50', 'resnet101']
lr         float                                   1e-05    0.01  True
iters      int                                     1       10
The get_hyperparameters method is implemented in the DetrLearner class.
The optimize method can be run without specifying the objective_function argument.


The output we get shows the hyperparameters that will be tuned, their type and their ranges. We will now start the optimization process. For this, we specify a *timeout* of 120, which means that hyperparameter tuning will be performed for a duration of two minutes. Also, set the progress bar argument to *True*, such that we are informed on the status of the process.

In [5]:
timeout = 120 # Optimize for 120 seconds

# Optimize
best_parameters = tuner.optimize(
    init_arguments=init_arguments,
    fit_arguments=fit_arguments,
    eval_arguments=eval_arguments,
    timeout=timeout,
    show_progress_bar=True,
)

/home/jelle/anaconda3/envs/opendr/lib/python3.6/site-packages/optuna/progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



0it [00:00, ?it/s]

/home/jelle/anaconda3/envs/opendr/lib/python3.6/site-packages/torch/_tensor.py:575: UserWarning:

floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630866422/work/aten/src/ATen/native/BinaryOps.cpp:467.)



And we are done! We can get the Optuna study from the tuner object using the *study* getter. Then, we will use the visualization tools from Optuna to visualize the results (you might need to install ipywidgets for this:```pip install "jupyterlab>=3" "ipywidgets>=7.6"```). 

In [11]:
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
study = tuner.study

In [12]:
plot_optimization_history(study)


In [13]:
plot_intermediate_values(study)

In [14]:
plot_contour(study)

In [15]:
plot_slice(study)

In [16]:
plot_param_importances(study)